###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Fiscalía General de la Nación

In [ ]:
import pandas as pd

###Archivo y2i3-im6h.csv

Contiene el eotal de Indiciados por delito según entradas de noticias criminales al Sistema Penal Oral Acusatorio en la Ley 906 de 2004 y Ley 1098 de 2006 desde hechos ocurridos en 2010.

In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/y2i3-im6h.csv?$limit=4166731")
df.head(3)

<ipython-input-3-678c84686827>:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/y2i3-im6h.csv?$limit=4166731")


,criminalidad,es_archivo,es_preclusion,estado,etapa_caso,ley,pais_hecho,departamento_hecho,municipio_hecho,seccional,...,delito,grupo_delito,consumado,sexo,grupo_etario,pais_nacimiento,aplica_lgbti,indigena,afrodescendiente,total_indiciados
0,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Valle del Cauca,CALI,DIRECCIÓN SECCIONAL DE CALI,...,HURTO ART. 239 C.P. MENOR CUANTIA,HURTO,NO APLICA,FEMENINO,SIN DATO,SIN DATO,NO,NO,NO,1
1,SI,NO,NO,ACTIVO,JUICIO,Ley 906,Colombia,Córdoba,MONTERÍA,DIRECCIÓN SECCIONAL DE CÓRDOBA,...,ACCESO CARNAL VIOLENTO. ART. 205 C.P.,DELITOS SEXUALES,NO APLICA,MASCULINO,Joven de 18 a 26 años.,Colombia,SI,NO,NO,1
2,SI,NO,NO,ACTIVO,INVESTIGACIÓN,Ley 906,Colombia,Antioquia,CÁCERES,DIRECCIÓN SECCIONAL DE ANTIOQUIA,...,EXTORSION. ART. 244 C.P MENOR CUANTIA,EXTORSION,NO APLICA,FEMENINO,Adulto entre 27 y 59 años.,Colombia,NO,NO,NO,2


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166731 entries, 0 to 4166730
Data columns (total 23 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   criminalidad        object
 1   es_archivo          object
 2   es_preclusion       object
 3   estado              object
 4   etapa_caso          object
 5   ley                 object
 6   pais_hecho          object
 7   departamento_hecho  object
 8   municipio_hecho     object
 9   seccional           object
 10  a_o_hechos          int64 
 11  a_o_entrada         object
 12  a_o_denuncia        object
 13  delito              object
 14  grupo_delito        object
 15  consumado           object
 16  sexo                object
 17  grupo_etario        object
 18  pais_nacimiento     object
 19  aplica_lgbti        object
 20  indigena            object
 21  afrodescendiente    object
 22  total_indiciados    int64 
dtypes: int64(2), object(21)
memory usage: 731.2+ MB


###- Eliminar indiciados por hechos que no hayan ocurrido en Colombia

In [ ]:
indicted_count = df[df['pais_hecho'] =='Colombia']
len(indicted_count)

4166696

### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['departamento_hecho', 'municipio_hecho', 'a_o_hechos',  'delito', 'grupo_delito', 'sexo', 'grupo_etario', 'total_indiciados']
indicted_count = indicted_count[relevant_cols]
indicted_count.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4166696 entries, 0 to 4166730
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   departamento_hecho  object
 1   municipio_hecho     object
 2   a_o_hechos          int64 
 3   delito              object
 4   grupo_delito        object
 5   sexo                object
 6   grupo_etario        object
 7   total_indiciados    int64 
dtypes: int64(2), object(6)
memory usage: 286.1+ MB


### Convertir columna a_o_hechos a tipo date

In [ ]:
# Convertir la columna 'a_o_hechos' a tipo datetime
indicted_count['a_o_hechos'] = pd.to_datetime(indicted_count['a_o_hechos'], format='%Y')
indicted_count['a_o_hechos'] = indicted_count['a_o_hechos'].dt.year

### Verificar valores nulos

In [ ]:
indicted_count.isnull().sum()

,0
departamento_hecho,0
municipio_hecho,0
a_o_hechos,0
delito,0
grupo_delito,0
sexo,0
grupo_etario,0
total_indiciados,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
for column in indicted_count.select_dtypes(include='object').columns:
    print(f"Categorías en la columna '{column}':")
    print(indicted_count[column].unique())
    print()

Categorías en la columna 'departamento_hecho':
['Valle del Cauca' 'Córdoba' 'Antioquia' 'Cundinamarca' 'La Guajira'
 'BOGOTÁ, D. C.' 'Atlántico' 'Santander' 'Huila' 'Tolima' 'Meta' 'Cesar'
 'Putumayo' 'Sucre' 'Bolívar' 'Caldas' 'Boyaca' 'Nariño' 'Risaralda'
 'Norte de Santander' 'Chocó' 'Magdalena' 'Caquetá' 'Quindío' 'Casanare'
 'Cauca' 'Vichada' 'Amazonas' 'Arauca' 'Guainía' 'Guaviare'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Vaupés']

Categorías en la columna 'municipio_hecho':
['CALI' 'MONTERÍA' 'CÁCERES' ... 'PAPUNAUA' 'BELÉN DE BAJIRÁ' 'PANA PANA']

Categorías en la columna 'delito':
['HURTO ART. 239 C.P. MENOR CUANTIA'
 'ACCESO CARNAL VIOLENTO. ART. 205 C.P.'
 'EXTORSION. ART. 244 C.P MENOR CUANTIA' ...
 'ABORTO SIN CONSENTIMIENTO ART. 123 C.P. ATENUADO POR ACCESO CARNAL VIOLENTO ART. 124 C.P.'
 'DISPOSICION DE BIEN PROPIO GRAVADO CON PRENDA. ART. 255 C.P. MENOR CUANTIA'
 'USO DE SOFTWARE MALICIOSO ART 269E LEY 1273 DE 2009, AGRAVADO POR SER COMETIDO POR EL R

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
categorical_columns = ['departamento_hecho', 'municipio_hecho', 'delito', 'grupo_delito', 'sexo', 'grupo_etario']

for col in categorical_columns:
   indicted_count[col] =indicted_count[col].apply(remove_accents_and_special_chars)

 - Codificación de algunas variables categóricas

## Agregar código del municipio a dataframe indicted_count para facilitar la georeferenciación

### -  Cargar los datos con códigos de departamentos y municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


### -  Asegurarse de que los nombres de los departamentos estén escritos de manera identica en cada datafram

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

- Comparar que los nombres de los departamentos y municipios en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    print(f"{label2} que no están en {label1}:")
    print(only_in_list2)
    print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(indicted_count['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en indicted_count", "Departamentos dept_mpios_codes")

Departamentos en indicted_count que no están en Departamentos dept_mpios_codes:
{'BOGOTA D. C.'}
Departamentos dept_mpios_codes que no están en Departamentos en indicted_count:
{'BOGOTA D.C.'}



- Solucionar discrepancias en los nombres de departamentos

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = { 'BOGOTA D. C.': 'BOGOTA D.C.'}

# Reemplazar los nombres incorrectos  el dataframe indicted_count
indicted_count['departamento_hecho'] = indicted_count['departamento_hecho'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas combinadas de departamento + municipio
compare_lists(indicted_count['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en indicted_count", "Departamentos dept_mpios_codes")

Departamentos en indicted_count que no están en Departamentos dept_mpios_codes:
set()
Departamentos dept_mpios_codes que no están en Departamentos en indicted_count:
set()



 - Comparar nombres de los departamentos combinados con los nombres de los municipios en los dos dataframe

In [ ]:
# Crear columnas combinadas de departamento + municipio
indicted_count['departamento_municipio'] = indicted_count['departamento_hecho'] + ' - ' + indicted_count['municipio_hecho']
dept_mpios_codes['departamento_municipio'] = dept_mpios_codes['dept_name'] + ' - ' + dept_mpios_codes['mupio_name']

# Comparar listas combinadas de departamento + municipio
compare_lists(indicted_count['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en indicted_count", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en indicted_count que no están en Departamentos y Municipios en dept_mpios_codes:
{'ANTIOQUIA - SAN PEDRO', 'NARINO - CUASPUD', 'NARINO - TUMACO', 'GUAINIA - BARRANCO MINAS', 'MAGDALENA - CERRO SAN ANTONIO', 'SUCRE - TOLU VIEJO', 'CAUCA - LOPEZ', 'BOLIVAR - CARTAGENA', 'BOYACA - GUICAN', 'VAUPES - PAPUNAUA', 'PUTUMAYO - LEGUIZAMO', 'GUAINIA - MAPIRIPANA', 'ANTIOQUIA - SAN VICENTE', 'CHOCO - BELEN DE BAJIRA', 'BOLIVAR - MOMPOS', 'ANTIOQUIA - DON MATIAS', 'CORDOBA - SAN ANDRES SOTAVENTO', 'CORDOBA - PURISIMA', 'CESAR - MANAURE', 'ANTIOQUIA - SANTAFE DE ANTIOQUIA', 'CUNDINAMARCA - SAN JUAN DE RIO SECO', 'CAUCA - EL BORDO', 'CAUCA - PIENDAMO', 'TOLIMA - MARIQUITA', 'CUNDINAMARCA - UBATE', 'MAGDALENA - CHIBOLO', 'NORTE DE SANTANDER - CUCUTA', 'CAUCA - SOTARA'}
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en indicted_count:
{'CORDOBA - PURISIMA DE LA CONCEPCION', 'ANTIOQUIA - SANTA FE DE ANTIOQUIA', 'CUNDINAMARCA - SAN J

 - Corregir nombres de los departamentos combinados con los nombres en dataframe indicted_count

In [ ]:
mapeo_manual = {
    'CUNDINAMARCA - SAN JUAN DE RIO SECO': 'CUNDINAMARCA - SAN JUAN DE RIOSECO',
    'TOLIMA - MARIQUITA': 'TOLIMA - SAN SEBASTIAN DE MARIQUITA',
    'CORDOBA - PURISIMA': 'CORDOBA - PURISIMA DE LA CONCEPCION',
    'BOLIVAR - MOMPOS': 'BOLIVAR - SANTA CRUZ DE MOMPOX',
    'NORTE DE SANTANDER - CUCUTA': 'NORTE DE SANTANDER - SAN JOSE DE CUCUTA',
    'CAUCA - EL BORDO': 'CAUCA - PATIA',
    'NARINO - TUMACO': 'NARINO - SAN ANDRES DE TUMACO',
    'CAUCA - SOTARA': 'CAUCA - SOTARA PAISPAMBA',
    'ANTIOQUIA - SAN PEDRO': 'ANTIOQUIA - SAN PEDRO DE LOS MILAGROS',
    'ANTIOQUIA - DON MATIAS': 'ANTIOQUIA - DONMATIAS',
    'CHOCO - BELEN DE BAJIRA': 'ANTIOQUIA - MUTATA',
    'BOLIVAR - CARTAGENA': 'BOLIVAR - CARTAGENA DE INDIAS',
    'NARINO - CUASPUD': 'NARINO - CUASPUD CARLOSAMA',
    'CORDOBA - SAN ANDRES SOTAVENTO': 'CORDOBA - SAN ANDRES DE SOTAVENTO',
    'MAGDALENA - CERRO SAN ANTONIO': 'MAGDALENA - CERRO DE SAN ANTONIO',
    'BOYACA - GUICAN': 'BOYACA - GUICAN DE LA SIERRA',
    'CUNDINAMARCA - UBATE': 'CUNDINAMARCA - VILLA DE SAN DIEGO DE UBATE',
    'CAUCA - LOPEZ': 'CAUCA - LOPEZ DE MICAY',
    'CESAR - MANAURE': 'CESAR - MANAURE BALCON DEL CESAR',
    'CAUCA - PIENDAMO': 'CAUCA - PIENDAMO - TUNIA',
    'GUAINIA - BARRANCO MINAS': 'GUAINIA - BARRANCOMINAS',
    'MAGDALENA - CHIBOLO': 'MAGDALENA - CHIVOLO',
    'SUCRE - TOLU VIEJO': 'SUCRE - SAN JOSE DE TOLUVIEJO',
    'GUAINIA - MAPIRIPANA': 'GUAINIA - PANA PANA',
    'ANTIOQUIA - SANTAFE DE ANTIOQUIA': 'ANTIOQUIA - SANTA FE DE ANTIOQUIA',
    'ANTIOQUIA - SAN VICENTE': 'ANTIOQUIA - SAN VICENTE FERRER',
    'VAUPES - PAPUNAUA': 'VAUPES - PAPUNAHUA',
    'PUTUMAYO - LEGUIZAMO': 'PUTUMAYO - PUERTO LEGUIZAMO',

}


indicted_count['departamento_municipio'] = indicted_count['departamento_municipio'].replace(mapeo_manual)

 - Comparar nombres de los departamentos combinados con los nombres de los municipios despues de corregir

In [ ]:
# Comparar listas combinadas de departamento + municipio
compare_lists(indicted_count['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en indicted_count", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en indicted_count que no están en Departamentos y Municipios en dept_mpios_codes:
set()
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en indicted_count:
set()



### Hacer merge entre los dataframes indicted-count y dept_mpios_codes

In [ ]:
# Realizamos el merge para obtener el código de municipio
indicted_count_with_codes = indicted_count.merge(dept_mpios_codes,
                                         left_on=['departamento_municipio'],
                                         right_on=['departamento_municipio'],
                                         how='left')

# Verificamos el resultado
indicted_count_with_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166696 entries, 0 to 4166695
Data columns (total 12 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   departamento_hecho      object
 1   municipio_hecho         object
 2   a_o_hechos              int32 
 3   delito                  object
 4   grupo_delito            object
 5   sexo                    object
 6   grupo_etario            object
 7   total_indiciados        int64 
 8   departamento_municipio  object
 9   dept_mpio_code          object
 10  dept_name               object
 11  mupio_name              object
dtypes: int32(1), int64(1), object(10)
memory usage: 365.6+ MB


In [ ]:
# No de filas por columna que no encontraron coincidencia
print(indicted_count_with_codes.isnull().sum())

departamento_hecho        0
municipio_hecho           0
a_o_hechos                0
delito                    0
grupo_delito              0
sexo                      0
grupo_etario              0
total_indiciados          0
departamento_municipio    0
dept_mpio_code            0
dept_name                 0
mupio_name                0
dtype: int64


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento_hecho', 'municipio_hecho', 'departamento_municipio', 'dept_name', 'mupio_name']
final_indicted_count = indicted_count_with_codes.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_indicted_count['source_id'] = 5

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'a_o_hechos': 'year_of_incident',
    'delito': 'crime_type',
    'grupo_delito': 'crime_group',
    'sexo': 'sex',
    'grupo_etario': 'age_group',
    'total_indiciados': 'total_indicted',
    'dept_mpio_code': 'dept_mpio_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_indicted_count.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_indicted_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166696 entries, 0 to 4166695
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   year_of_incident  int32 
 1   crime_type        object
 2   crime_group       object
 3   sex               object
 4   age_group         object
 5   total_indicted    int64 
 6   dept_mpio_code    object
 7   source_id         int64 
dtypes: int32(1), int64(2), object(5)
memory usage: 238.4+ MB


### Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_indicted_count.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/indicted_count.csv', index=False)